In [6]:
# %% [markdown]
# ## Classical–Quantum comparison helper cell
#
# Solves A x = b with the NumPyLinearSolver (classical)
# and with HHL (quantum), then prints the results in
# a uniform, easy‑to‑compare format.

# %%
import numpy as np
from qiskit_aer import AerSimulator                     # backend
from qiskit.quantum_info import Statevector             # to unwrap the HHL state    # classical reference
from QLS.hhl import HHL                                     # Qiskit‑based HHL implementation
from QLS.numpy_linear_solver import NumPyLinearSolver
# ----- problem definition ----------------------------------------------------
A = np.array([[1, -1/3],
              [-1/3, 1]], dtype=float)
b = np.array([1, 0], dtype=float)

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b / np.linalg.norm(b)           # renormalise for fair comparison
)

# ----- quantum (HHL) solution -----------------------------------------------
backend = AerSimulator()
hhl_solver = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b)

# Helper: extract the two amplitudes that encode |x⟩ from the simulated state
def extract_solution(result):
    """Return the real 2‑component solution vector encoded in result.state."""
    sv = Statevector(result.state).data
    # For 1 work qubit, 2 phase‑est. qubits, 1 ancilla:
    # amplitudes |10000⟩ (decimal 16) and |10001⟩ (17) hold x₁ and x₂
    amp = sv[16:18].real
    return result.euclidean_norm * amp / np.linalg.norm(amp)

x_classical = classical_res.state                    # already a NumPy vector
x_quantum   = extract_solution(quantum_res)          # recovered from circuit

# ----- print side‑by‑side comparison -----------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum)
print()
print("Classical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print()
print("ℓ₂‑norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))

Classical solution vector: [1.125 0.375]
Quantum   solution vector: [1.12851015 0.37617005]

Classical Euclidean norm : 1.1858541225631423
Quantum   Euclidean norm : 1.1895541444171425

ℓ₂‑norm of (x_classical − x_quantum): 0.0037000218540000703


In [51]:
# %% [markdown]
# ## Classical–Quantum comparison (4 × 4 linear system)
#
# We now solve a 4×4 tridiagonal Toeplitz system with both
# NumPyLinearSolver (classical reference) and HHL (quantum),
# then print the two solution vectors, their Euclidean norms
# and the ℓ₂‑distance between them.

# %%
import numpy as np
from scipy.sparse import diags
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from QLS.numpy_linear_solver import NumPyLinearSolver   # classical
from QLS.hhl                   import HHL              # quantum HHL

# ----- problem definition ----------------------------------------------------
NUM_WORK_QUBITS = 2                 # log₂(4) → 4‑dimensional vector
DIM              = 2 ** NUM_WORK_QUBITS

a_diag = 1
b_off  = -1/3
A = diags([b_off, a_diag, b_off], [-1, 0, 1],
          shape=(DIM, DIM)).toarray()
print("Matrix A:\n", A)
b_vec = np.array([1] + [0]*(DIM - 1), dtype=float)

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)     # re‑normalise for fair comparison
)

# ----- quantum (HHL) solution -----------------------------------------------
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    """Return the real 2ⁿ solution vector encoded in result.state.
       Assumes the (single) ancilla qubit is the MSB, and we take the
       amplitudes where ancilla = 1 and all phase‑est. qubits are 0."""
    sv            = Statevector(result.state).data
    total_qubits  = int(np.log2(len(sv)))
    base_index    = 1 << (total_qubits - 1)          # ancilla bit set to 1
    amps          = np.array([sv[base_index + i].real
                              for i in range(2 ** n_work_qubits)])
    # renormalise with the Euclidean norm returned by HHL
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ----- side‑by‑side printout -------------------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum)
print()
print("Classical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print()
print("ℓ₂‑norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))

Matrix A:
 [[ 1.         -0.33333333  0.          0.        ]
 [-0.33333333  1.         -0.33333333  0.        ]
 [ 0.         -0.33333333  1.         -0.33333333]
 [ 0.          0.         -0.33333333  1.        ]]
Classical solution vector: [1.14545455 0.43636364 0.16363636 0.05454545]
Quantum   solution vector: [1.13665223 0.44812171 0.16349669 0.09135507]

Classical Euclidean norm : 1.237833351044751
Quantum   Euclidean norm : 1.2360696878097173

ℓ₂‑norm of (x_classical − x_quantum): 0.039632059354351254


In [65]:
import numpy as np
from scipy.sparse import diags
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from QLS.numpy_linear_solver import NumPyLinearSolver   # classical
from QLS.hhl                   import HHL              # quantum HHL

# ----- problem definition ----------------------------------------------------
NUM_WORK_QUBITS = 3                 # log₂(4) → 4-dimensional vector
DIM              = 2 ** NUM_WORK_QUBITS

# Option 1: random REAL Hermitian (symmetric) matrix                # for reproducibility (optional)
M_real = np.random.randn(DIM, DIM)
A = (M_real + M_real.T) / 2         # A is now real symmetric


print("Random Hermitian A:\n", A)

# right-hand side
b_vec = np.array([1] + [0]*(DIM - 1), dtype=complex if np.iscomplexobj(A) else float)

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)     # re-normalise for fair comparison
)

# ----- quantum (HHL) solution -----------------------------------------------
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    """Return the real (or complex) 2ⁿ solution vector encoded in result.state.
       Assumes the (single) ancilla qubit is the MSB, and we take the
       amplitudes where ancilla = 1 and all phase-est. qubits are 0."""
    sv            = Statevector(result.state).data
    total_qubits  = int(np.log2(len(sv)))
    base_index    = 1 << (total_qubits - 1)          # ancilla bit set to 1
    amps          = np.array([sv[base_index + i]
                              for i in range(2 ** n_work_qubits)])
    # renormalise with the Euclidean norm returned by HHL
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ----- side-by-side printout -------------------------------------------------
print("\nClassical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum)
print()
print("Classical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print()
print("ℓ₂-norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))

Random Hermitian A:
 [[ 0.80948549  1.09527061  0.02804151  0.43796862 -1.37557357  0.80217717
  -1.83526126  0.29760326]
 [ 1.09527061  0.98906034 -0.64971441 -0.22841273 -1.21546089  0.60855977
   0.57148164 -0.03812055]
 [ 0.02804151 -0.64971441  1.38626644 -0.52666347  1.42224222 -0.0369264
   0.0334028   0.42373953]
 [ 0.43796862 -0.22841273 -0.52666347  2.2647943   0.40577574 -0.39210834
  -0.59235973  0.76717905]
 [-1.37557357 -1.21546089  1.42224222  0.40577574  0.58909137  1.27694387
   0.05289406 -0.92592936]
 [ 0.80217717  0.60855977 -0.0369264  -0.39210834  1.27694387  0.76273567
  -0.40104834  0.71140493]
 [-1.83526126  0.57148164  0.0334028  -0.59235973  0.05289406 -0.40104834
   1.41619788 -0.89473103]
 [ 0.29760326 -0.03812055  0.42373953  0.76717905 -0.92592936  0.71140493
  -0.89473103 -0.11459239]]

Classical solution vector: [-10.58994428  -4.92827398  -2.30368384  -3.12573546  -7.46484015
   6.86106201   3.25454963  22.19228922]
Quantum   solution vector: [-10.9714

In [2]:
#!/usr/bin/env python3
"""Classical vs‑Quantum (HHL) linear‑system solver demo on a GPU simulator.

* Works with real **or** complex Hermitian matrices.
* Uses Qiskit Aer’s GPU state‑vector backend.
"""

import numpy as np
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

# ---------------------------------------------------------------------------
#  Qiskit‑based linear‑system solvers
# ---------------------------------------------------------------------------
from numpy_linear_solver import NumPyLinearSolver   # classical reference
from hhl                   import HHL              # quantum HHL algorithm

# ------------------------- 1.  Problem definition ---------------------------
NUM_WORK_QUBITS = 4               # 4 work‑qubits  →  16‑dimensional system
DIM              = 2 ** NUM_WORK_QUBITS

np.random.seed(42)                # reproducibility (optional)

# ---- Option A: random REAL symmetric (Hermitian) matrix --------------------
M_real = np.random.randn(DIM, DIM)
A      = (M_real + M_real.T) / 2          # A is real symmetric / Hermitian

# ---- Option B: random COMPLEX Hermitian matrix -----------------------------
# Uncomment to test with a complex matrix.
# M_c = np.random.randn(DIM, DIM) + 1j*np.random.randn(DIM, DIM)
# A  = (M_c + M_c.conj().T) / 2           # A is complex Hermitian

print("Random Hermitian A (shape {}):\n".format(A.shape), A)

# Right‑hand side vector  b
b_vec = np.array([1] + [0]*(DIM - 1),
                 dtype=complex if np.iscomplexobj(A) else float)

# ------------------------- 2.  Classical solution ---------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)      # renormalise for fair comparison
)

# ------------------------- 3.  Quantum (HHL) solution -----------------------
# GPU backend: state‑vector simulator on CUDA device
backend_gpu = AerSimulator(method='statevector',
                           device='GPU',
                           precision='double')      # or 'single' for speed
print("Using GPU backend:", backend_gpu.name())

hhl_gpu = HHL(epsilon=1e-3, quantum_instance=backend_gpu)
quantum_res = hhl_gpu.solve(A, b_vec)

# Helper ─────────────────────────────────────────────────────────────────────
def extract_solution(result, n_work_qubits: int):
    """
    Recover the length‑2ⁿ solution vector encoded in the simulated HHL state.

    Assumes:
      * single ancilla qubit is the MSB (index 0 after little‑endian reversal);
      * all QPE (phase estimation) qubits are 0 in the desired branch.
    """
    sv           = Statevector(result.state).data
    total_qubits = int(np.log2(len(sv)))
    base_index   = 1 << (total_qubits - 1)          # ancilla bit set to 1
    amps = np.array([sv[base_index + i]
                     for i in range(2 ** n_work_qubits)],
                    dtype=sv.dtype)

    # Renormalise with the Euclidean norm HHL already computed
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ------------------------- 4.  Results --------------------------------------
print("\n────────────────────────────────────────────────────────")
print("Classical solution vector:\n", x_classical)
print("\nQuantum   solution vector:\n", x_quantum)
print("\nClassical Euclidean norm :", classical_res.euclidean_norm)
print("Quantum   Euclidean norm :", quantum_res.euclidean_norm)
print("\nℓ₂‑norm of (x_classical − x_quantum):",
      np.linalg.norm(x_classical - x_quantum))
print("────────────────────────────────────────────────────────")

ModuleNotFoundError: No module named 'numpy_linear_solver'

In [49]:
# %% [markdown]
# ## Linear‑regression via normal equations – classical vs quantum HHL

# %%
import numpy as np
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from QLS.numpy_linear_solver import NumPyLinearSolver   # classical reference
from QLS.hhl                   import HHL              # quantum HHL

# ---------------------- 1.  Synthetic regression data -----------------------
n_samples   = 50          # rows of X
n_features  = 4           # columns of X  →  4 × 4 system  (2 work‑qubits)

X = np.random.randn(n_samples, n_features)
true_theta = np.array([2.0, -1.0, 0.5, 3.0])
y = X @ true_theta + 0.2*np.random.randn(n_samples)   # add a bit of noise

# Normal‑equation components
A = X.T @ X                  # real symmetric / Hermitian
b_vec = X.T @ y

# For HHL we need a power‑of‑two dimension (n_features=4 → OK)
NUM_WORK_QUBITS = int(np.log2(n_features))

print("Normal‑equation matrix A:\n", A)
print("Right‑hand side b:\n", b_vec)

# ---------------------- 2.  Classical reference solution --------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)    # renormalise (HHL convention)
)

theta_classical = classical_res.state        # already a NumPy vector

# ---------------------- 3.  Quantum (HHL on GPU) ----------------------------
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)
# Helper: recover the 4‑component θ from the quantum state
def extract_solution(result, n_work_qubits):
    sv           = Statevector(result.state).data
    total_qubits = int(np.log2(len(sv)))
    base         = 1 << (total_qubits - 1)             # ancilla = 1
    amps = np.array([sv[base + i] for i in range(2**n_work_qubits)],
                    dtype=sv.dtype)
    return result.euclidean_norm * amps / np.linalg.norm(amps)

theta_quantum = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ---------------------- 4.  Compare the two models --------------------------
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

mse_classical = mse(y, X @ theta_classical)
mse_quantum   = mse(y, X @ theta_quantum)

print("\n────────────────────────────────────────────────────────")
print("Parameter vector θ (classical):", theta_classical)
print("Parameter vector θ (quantum)  :", theta_quantum)
print("\nEuclidean norm θ (classical):", classical_res.euclidean_norm)
print("Euclidean norm θ (quantum)  :", quantum_res.euclidean_norm)
print("\nTraining‑set MSE (classical):", mse_classical)
print("Training‑set MSE (quantum)  :", mse_quantum)
print("\n‖θ_classical − θ_quantum‖₂   :", np.linalg.norm(theta_classical
                                                      - theta_quantum))
print("────────────────────────────────────────────────────────")

Normal‑equation matrix A:
 [[ 42.15615712   2.78070712  -1.26977861  -5.95576291]
 [  2.78070712  46.67813908  21.11230549   1.92787183]
 [ -1.26977861  21.11230549  69.56974676 -11.55680913]
 [ -5.95576291   1.92787183 -11.55680913  46.723258  ]]
Right‑hand side b:
 [ 61.19368994 -22.72857048 -25.70028401 122.02548051]

────────────────────────────────────────────────────────
Parameter vector θ (classical): [ 0.01386586 -0.00660976  0.00318651  0.02138305]
Parameter vector θ (quantum)  : [ 0.01394874-2.72041843e-16j -0.00671638+4.24942311e-16j
  0.00308084-2.42475392e-17j  0.02044903+5.99774832e-16j]

Euclidean norm θ (classical): 0.02652055201809843
Euclidean norm θ (quantum)  : 0.025832756698476955

Training‑set MSE (classical): 9.822580263405404
Training‑set MSE (quantum)  : (9.82670216281081-1.8879975378144103e-15j)

‖θ_classical − θ_quantum‖₂   : 0.0009496301285800121
────────────────────────────────────────────────────────


In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

from QLS.numpy_linear_solver import NumPyLinearSolver   # classical
from QLS.hhl import HHL              # quantum HHL

# ----- user parameters ------------------------------------------------------
make_hermitian = True       # True → Hermitian SPD; False → general non-Hermitian
target_kappa   = 1e1        # desired condition number κ(A)
density        = 0.8        # fraction of nonzero entries (0<density≤1)
noise_level    = 1e1       # relative off-diag noise for SPD case

# ----- problem definition ---------------------------------------------------
NUM_WORK_QUBITS = 3                 # ⇒ 4-dimensional system
DIM             = 2 ** NUM_WORK_QUBITS

# ----- helpers ---------------------------------------------------------------
def generate_sparse_spd(n, kappa, density, noise_level):
    """Hermitian SPD with log-spaced eigenvalues and random sparse off-diag noise."""
    # 1) log-spaced eigenvalues
    eigs = np.logspace(0, np.log10(kappa), n)
    # 2) diagonal entries
    rows = np.arange(n); cols = rows; data = eigs
    A = coo_matrix((data, (rows, cols)), shape=(n, n))
    # 3) add symmetric off-diagonal noise
    total = n*n
    nnz   = int(density*total)
    off   = max(nnz - n, 0)
    off  -= off % 2
    half  = off//2
    if half>0:
        i = np.random.randint(0,n,half*2)
        j = np.random.randint(0,n,half*2)
        mask = (i!=j)
        i,j = i[mask][:half], j[mask][:half]
        eps  = noise_level * eigs.min()
        vals = np.random.uniform(-eps, eps, size=half)
        A   = A + coo_matrix((vals,(i,j)),shape=(n,n))\
                + coo_matrix((vals,(j,i)),shape=(n,n))
    return A.tocsr()

def generate_general(n, kappa, density):
    """General (non-Hermitian) matrix with approx κ via SVD, then sparsified."""
    # 1) U, V random orthonormal
    U,_  = np.linalg.qr(np.random.randn(n,n))
    V,_  = np.linalg.qr(np.random.randn(n,n))
    # 2) singular values
    s    = np.logspace(0, np.log10(kappa), n)
    A    = U @ np.diag(s) @ V.T
    # 3) sparsify by zeroing random entries
    mask = (np.random.rand(n,n) < density)
    A   *= mask
    return A

# ----- build A ----------------------------------------------------------------
if make_hermitian:
    A_sparse = generate_sparse_spd(DIM, target_kappa, density, noise_level)
    A        = A_sparse.toarray()
else:
    A = generate_general(DIM, target_kappa, density)

# ----- checks & print ---------------------------------------------------------
is_herm = np.allclose(A, A.conj().T, atol=1e-12)
cond_A  = np.linalg.cond(A)
nnz     = np.count_nonzero(A)
print(f"A (dim={DIM}×{DIM}), Hermitian? {is_herm}, κ(A) ≈ {cond_A:.3e}, "
      f"sparsity={nnz}/{DIM*DIM}={nnz/(DIM*DIM):.2%}\n")
#print("A =\n", A)   # uncomment to see the full matrix

# ----- right-hand side -------------------------------------------------------
b_vec = np.zeros(DIM, dtype=complex if np.iscomplexobj(A) else float)
b_vec[0] = 1

# ----- classical solution ----------------------------------------------------
classical_res = NumPyLinearSolver().solve(
    A,
    b_vec / np.linalg.norm(b_vec)
)

# ----- quantum (HHL) solution ------------------------------------------------
backend     = AerSimulator()
hhl_solver  = HHL(epsilon=1e-3, quantum_instance=backend)
quantum_res = hhl_solver.solve(A, b_vec)

def extract_solution(result, n_work_qubits: int) -> np.ndarray:
    sv           = Statevector(result.state).data
    total_qubits = int(np.log2(len(sv)))
    base_index   = 1 << (total_qubits - 1)
    amps         = np.array([sv[base_index + i]
                             for i in range(2 ** n_work_qubits)])
    return result.euclidean_norm * amps / np.linalg.norm(amps)

x_classical = classical_res.state
x_quantum   = extract_solution(quantum_res, NUM_WORK_QUBITS)

# ----- results ---------------------------------------------------------------
print("Classical solution vector:", x_classical)
print("Quantum   solution vector:", x_quantum, "\n")
print("Classical Euclidean norm:", classical_res.euclidean_norm)
print("Quantum   Euclidean norm:", quantum_res.euclidean_norm, "\n")
print("‖x_classical − x_quantum‖₂ =",
      np.linalg.norm(x_classical - x_quantum))

A (dim=8×8), Hermitian? True, κ(A) ≈ 2.094e+02, sparsity=36/64=56.25%

Classical solution vector: [-0.40779945  0.47110686 -0.92055067 -0.06433487 -0.94601421 -0.11507417
  0.31250642 -0.39604582]
Quantum   solution vector: [-0.40845471-3.64703312e-12j  0.47194398+4.13272839e-12j
 -0.92202881-8.20669199e-12j -0.0644256 -5.89375498e-13j
 -0.94760835-8.38038475e-12j -0.11522446-1.05756707e-12j
  0.31312433+2.72770353e-12j -0.39666573-3.56565969e-12j] 

Classical Euclidean norm: 1.5499974433064758
Quantum   Euclidean norm: 1.5525730078371272 

‖x_classical − x_quantum‖₂ = 0.0025793829385511377


In [15]:
import numpy as np
np.__version__

'2.2.3'